# GitHub MCP Agent with LangGraph & LangChain

This notebook demonstrates how to build an autonomous agent using LangGraph and LangChain that can:
- Use configurable LLM connections (Ollama and Gemini)
- Perform basic operations (starting with addition)
- Integrate with GitHub using Model Context Protocol (MCP)

## Table of Contents
1. Setup Project Environment with UV
2. Install Required Dependencies  
3. Configure LLM Connections (Ollama and Gemini)
4. Import Required Libraries
5. Create Basic Addition Tool
6. Build LangGraph Agent Architecture
7. Define Agent State and Nodes
8. Create Agent Workflow
9. Test Basic Addition Agent
10. GitHub MCP Integration Setup
11. Extend Agent with GitHub Capabilities

## 1. Setup Project Environment with UV

We're using UV for fast Python package management. The project structure has been initialized with the following components:

- `pyproject.toml` - Project configuration and dependencies
- `.env.example` - Template for environment variables
- `.env` - Local environment configuration
- `agent_notebook.ipynb` - This notebook

Key benefits of using UV:
- Faster dependency resolution and installation
- Better dependency management
- Virtual environment management
- Compatible with pip and PyPI

In [ ]:
# Verify UV installation and show project structure
import subprocess
import os
from pathlib import Path

# Check UV version
try:
    result = subprocess.run(['uv', '--version'], capture_output=True, text=True)
    print(f"UV Version: {result.stdout.strip()}")
except FileNotFoundError:
    print("UV is not installed. Please install UV first.")

# Show current project structure
project_root = Path.cwd()
print(f"\nProject Root: {project_root}")
print("\nProject Structure:")
for item in sorted(project_root.iterdir()):
    if item.name.startswith('.') and item.name not in ['.env', '.env.example']:
        continue
    print(f"  {item.name}")

# Show pyproject.toml content
pyproject_path = project_root / "pyproject.toml"
if pyproject_path.exists():
    print(f"\nContents of pyproject.toml:")
    with open(pyproject_path, 'r') as f:
        print(f.read())

: 

## 2. Install Required Dependencies

Our project includes the following key dependencies:

- **LangChain & LangGraph**: Core framework for building the agent
- **Ollama**: Local LLM integration
- **Google Generative AI**: Gemini API integration  
- **MCP**: Model Context Protocol for GitHub integration
- **Jupyter**: For interactive development
- **Python-dotenv**: Environment variable management

Dependencies have been pre-installed via `uv sync`.

In [ ]:
# Verify key packages are installed
import pkg_resources

key_packages = [
    'langchain',
    'langchain-core',
    'langgraph',
    'langchain-ollama',
    'langchain-google-genai',
    'jupyter',
    'python-dotenv',
    'pydantic',
    'httpx',
    'mcp'
]

print("📦 Installed Key Packages:")
print("=" * 50)

for package in key_packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"✅ {package:<25} v{version}")
    except pkg_resources.DistributionNotFound:
        print(f"❌ {package:<25} NOT INSTALLED")

print("=" * 50)
print("✨ Package verification complete!")

## 3. Configure LLM Connections (Ollama and Gemini)

We'll create a configurable system that allows switching between different LLM providers:

- **Ollama**: For local LLM inference (privacy-focused, offline capable)
- **Gemini**: For Google's powerful cloud-based models

The configuration will be managed through environment variables, making it easy to switch providers without code changes.

In [ ]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import Literal, Optional
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables
load_dotenv()

class LLMConfig(BaseModel):
    """Configuration for LLM connections"""
    provider: Literal["ollama", "gemini"] = "ollama"

    # Ollama configuration
    ollama_base_url: str = "http://localhost:11434"
    ollama_model: str = "llama2"

    # Gemini configuration
    google_api_key: Optional[str] = None
    gemini_model: str = "gemini-pro"

def load_llm_config() -> LLMConfig:
    """Load LLM configuration from environment variables"""
    return LLMConfig(
        provider=os.getenv("DEFAULT_LLM_PROVIDER", "ollama"),
        ollama_base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
        ollama_model=os.getenv("OLLAMA_MODEL", "llama2"),
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        gemini_model=os.getenv("GEMINI_MODEL", "gemini-pro")
    )

def create_llm(config: LLMConfig):
    """Create LLM instance based on configuration"""
    if config.provider == "ollama":
        print(f"🦙 Initializing Ollama with model: {config.ollama_model}")
        return ChatOllama(
            base_url=config.ollama_base_url,
            model=config.ollama_model,
            temperature=0.1
        )
    elif config.provider == "gemini":
        if not config.google_api_key:
            raise ValueError("Google API key is required for Gemini provider")
        print(f"🤖 Initializing Gemini with model: {config.gemini_model}")
        return ChatGoogleGenerativeAI(
            model=config.gemini_model,
            google_api_key=config.google_api_key,
            temperature=0.1
        )
    else:
        raise ValueError(f"Unsupported LLM provider: {config.provider}")

# Initialize configuration
config = load_llm_config()
print(f"📋 LLM Configuration:")
print(f"  Provider: {config.provider}")
print(f"  Ollama URL: {config.ollama_base_url}")
print(f"  Ollama Model: {config.ollama_model}")
print(f"  Gemini Model: {config.gemini_model}")
print(f"  Google API Key: {'✅ Set' if config.google_api_key else '❌ Not set'}")

## 4. Import Required Libraries

Now let's import all the necessary libraries for building our LangGraph agent with LangChain integration.

In [ ]:
# Core LangChain and LangGraph imports
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import BaseRetriever

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# Standard library imports
import json
from typing import Annotated, Dict, Any, List, Sequence, TypedDict
import operator

# Additional utilities
from pydantic import BaseModel, Field
import asyncio

print("📚 All required libraries imported successfully!")

## 5. Create Basic Addition Tool

Let's start with a simple tool that can add two numbers. This will serve as our foundation before extending to more complex GitHub operations.

In [ ]:
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together.

    Args:
        a: First number to add
        b: Second number to add

    Returns:
        The sum of a and b
    """
    result = a + b
    print(f"🔢 Adding {a} + {b} = {result}")
    return result

@tool
def multiply_numbers(x: int, y: int) -> int:
    """Multiply two numbers together.

    Args:
        x: First number to multiply
        y: Second number to multiply

    Returns:
        The product of x and y
    """
    result = x * y
    print(f"✖️ Multiplying {x} × {y} = {result}")
    return result

@tool
def get_calculation_info() -> str:
    """Get information about available mathematical operations.

    Returns:
        Information about supported operations
    """
    info = """
    📊 Available Mathematical Operations:
    - Addition: Use add_numbers(a, b) to add two integers
    - Multiplication: Use multiply_numbers(x, y) to multiply two integers

    Example usage:
    - "Add 5 and 3" → add_numbers(5, 3) → 8
    - "Multiply 4 by 6" → multiply_numbers(4, 6) → 24
    """
    return info

# Create tools list
basic_tools = [add_numbers, multiply_numbers, get_calculation_info]

print("🔧 Basic mathematical tools created:")
for tool in basic_tools:
    print(f"  - {tool.name}: {tool.description}")
print(f"\n✅ Total tools available: {len(basic_tools)}")

## 6. Build LangGraph Agent Architecture

Now we'll create the core agent architecture using LangGraph. This includes:

- **State Management**: Defining how the agent tracks conversation and tool usage
- **LLM Integration**: Connecting our configurable LLM to the agent
- **Tool Binding**: Making our tools available to the agent

In [ ]:
# Initialize the LLM with our configuration
try:
    llm = create_llm(config)
    print(f"✅ LLM initialized successfully: {config.provider}")
except Exception as e:
    print(f"❌ Failed to initialize LLM: {e}")
    print("🔄 Falling back to a mock LLM for demonstration")
    # Create a simple mock for demo purposes
    class MockLLM:
        def invoke(self, messages):
            return AIMessage(content="I would process this with a real LLM")
        def bind_tools(self, tools):
            return self
    llm = MockLLM()

# Bind tools to LLM
llm_with_tools = llm.bind_tools(basic_tools)

print(f"🔧 LLM bound with {len(basic_tools)} tools")
print("🏗️ Agent architecture foundation ready!")

## 7. Define Agent State and Nodes

The agent state will track the conversation messages and any additional context needed for operations.

In [ ]:
# Define the agent state
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

class BaseMessage:
    """Base message class for compatibility"""
    def __init__(self, content: str, type: str = "human"):
        self.content = content
        self.type = type

# Define the assistant node (LLM call)
def call_model(state: AgentState):
    """Call the LLM with the current conversation state"""
    print("🤖 Assistant is thinking...")

    # Get the messages from state
    messages = state["messages"]

    try:
        # Call the LLM
        response = llm_with_tools.invoke(messages)
        print(f"💭 Assistant response: {response.content}")

        # Return the response to add to the state
        return {"messages": [response]}
    except Exception as e:
        print(f"❌ Error calling model: {e}")
        # Return an error message
        error_response = AIMessage(content=f"I encountered an error: {str(e)}")
        return {"messages": [error_response]}

print("🏗️ Agent state and nodes defined:")
print("  - AgentState: Tracks conversation messages")
print("  - call_model: Handles LLM interactions")
print("✅ Ready to build the workflow!")

## 8. Create Agent Workflow

Now we'll create the complete LangGraph workflow that orchestrates the conversation flow between the LLM and tools.

In [ ]:
# Create the workflow
workflow = StateGraph(AgentState)

# Create tool node
tool_node = ToolNode(basic_tools)

# Add nodes to the workflow
workflow.add_node("assistant", call_model)
workflow.add_node("tools", tool_node)

# Set the entry point
workflow.set_entry_point("assistant")

# Add conditional edges
workflow.add_conditional_edges(
    "assistant",
    tools_condition,  # This decides whether to call tools or end
    {
        "tools": "tools",
        "__end__": END,
    }
)

# Add edge from tools back to assistant
workflow.add_edge("tools", "assistant")

# Add memory for conversation persistence
memory = MemorySaver()

# Compile the workflow
try:
    agent = workflow.compile(checkpointer=memory)
    print("✅ Agent workflow compiled successfully!")
    print("🔄 Workflow structure:")
    print("  1. User input → Assistant (LLM)")
    print("  2. Assistant decides: Use tools or respond")
    print("  3. If tools needed: Execute → Return to Assistant")
    print("  4. Final response to user")
except Exception as e:
    print(f"❌ Error compiling workflow: {e}")
    agent = None

## 9. Test Basic Addition Agent

Let's test our agent with some basic mathematical operations to ensure everything is working correctly.

In [ ]:
def test_agent(query: str, thread_id: str = "test-thread"):
    """Test the agent with a query"""
    print(f"🧪 Testing: '{query}'")
    print("=" * 50)

    if agent is None:
        print("❌ Agent not available - workflow compilation failed")
        return

    try:
        # Create the input message
        input_message = HumanMessage(content=query)

        # Run the agent
        config = {"configurable": {"thread_id": thread_id}}

        # Stream the agent's response
        for chunk in agent.stream(
            {"messages": [input_message]},
            config=config
        ):
            print(f"📤 Chunk: {chunk}")

    except Exception as e:
        print(f"❌ Error during agent execution: {e}")

    print("=" * 50)
    print()

# Test cases
test_queries = [
    "Add 15 and 27",
    "What's 8 multiplied by 9?",
    "Can you tell me what mathematical operations you can perform?",
    "Calculate 100 plus 200, then multiply the result by 3"
]

print("🚀 Starting Agent Tests...")
print()

for i, query in enumerate(test_queries, 1):
    print(f"Test {i}:")
    test_agent(query, f"test-thread-{i}")

print("✅ All tests completed!")

## 10. GitHub MCP Integration Setup

Now let's set up the GitHub Model Context Protocol (MCP) integration. This will allow our agent to interact with GitHub repositories, issues, pull requests, and more.

**Note**: This section requires:
1. A GitHub personal access token
2. Repository permissions appropriate for your use case
3. The MCP GitHub server setup

In [ ]:
# GitHub MCP Configuration
import httpx
from typing import Optional

class GitHubMCPConfig(BaseModel):
    """Configuration for GitHub MCP integration"""
    github_token: Optional[str] = None
    github_repo: Optional[str] = None
    base_url: str = "https://api.github.com"

def load_github_config() -> GitHubMCPConfig:
    """Load GitHub configuration from environment"""
    return GitHubMCPConfig(
        github_token=os.getenv("GITHUB_TOKEN"),
        github_repo=os.getenv("GITHUB_REPO"),
        base_url=os.getenv("GITHUB_BASE_URL", "https://api.github.com")
    )

# GitHub Tools for the agent
@tool
def get_repo_info(repo_name: str) -> str:
    """Get basic information about a GitHub repository.

    Args:
        repo_name: Repository name in format 'owner/repo'

    Returns:
        Repository information as a formatted string
    """
    github_config = load_github_config()

    if not github_config.github_token:
        return "❌ GitHub token not configured. Please set GITHUB_TOKEN environment variable."

    try:
        headers = {
            "Authorization": f"token {github_config.github_token}",
            "Accept": "application/vnd.github.v3+json"
        }

        url = f"{github_config.base_url}/repos/{repo_name}"

        with httpx.Client() as client:
            response = client.get(url, headers=headers)

        if response.status_code == 200:
            repo_data = response.json()
            info = f"""
            📁 Repository: {repo_data['full_name']}
            📝 Description: {repo_data.get('description', 'No description')}
            ⭐ Stars: {repo_data['stargazers_count']}
            🍴 Forks: {repo_data['forks_count']}
            📊 Language: {repo_data.get('language', 'Not specified')}
            🔗 URL: {repo_data['html_url']}
            """
            return info
        else:
            return f"❌ Failed to fetch repository info: {response.status_code}"

    except Exception as e:
        return f"❌ Error fetching repository info: {str(e)}"

@tool
def list_repo_issues(repo_name: str, limit: int = 5) -> str:
    """List recent issues from a GitHub repository.

    Args:
        repo_name: Repository name in format 'owner/repo'
        limit: Maximum number of issues to return (default: 5)

    Returns:
        List of recent issues as a formatted string
    """
    github_config = load_github_config()

    if not github_config.github_token:
        return "❌ GitHub token not configured."

    try:
        headers = {
            "Authorization": f"token {github_config.github_token}",
            "Accept": "application/vnd.github.v3+json"
        }

        url = f"{github_config.base_url}/repos/{repo_name}/issues"
        params = {"state": "open", "per_page": limit}

        with httpx.Client() as client:
            response = client.get(url, headers=headers, params=params)

        if response.status_code == 200:
            issues = response.json()

            if not issues:
                return f"📭 No open issues found in {repo_name}"

            result = f"📋 Recent Issues in {repo_name}:\\n\\n"
            for issue in issues:
                result += f"#{issue['number']}: {issue['title']}\\n"
                result += f"   👤 Created by: {issue['user']['login']}\\n"
                result += f"   📅 Created: {issue['created_at'][:10]}\\n"
                result += f"   🏷️ Labels: {', '.join([label['name'] for label in issue['labels']])}\\n\\n"

            return result
        else:
            return f"❌ Failed to fetch issues: {response.status_code}"

    except Exception as e:
        return f"❌ Error fetching issues: {str(e)}"

# Create GitHub tools list
github_tools = [get_repo_info, list_repo_issues]

# Load GitHub configuration
github_config = load_github_config()
print("🐙 GitHub MCP Configuration:")
print(f"  Token: {'✅ Set' if github_config.github_token else '❌ Not set'}")
print(f"  Default Repo: {github_config.github_repo or 'Not set'}")
print(f"  Base URL: {github_config.base_url}")
print(f"\\n🔧 GitHub tools available: {len(github_tools)}")
for tool in github_tools:
    print(f"  - {tool.name}: {tool.description}")

## 11. Extend Agent with GitHub Capabilities

Now let's create an enhanced version of our agent that includes both mathematical operations and GitHub repository management capabilities.

In [ ]:
# Combine all tools for the enhanced agent
all_tools = basic_tools + github_tools

print(f"🔧 Enhanced Agent Tools ({len(all_tools)} total):")
print("\\nMathematical Operations:")
for tool in basic_tools:
    print(f"  - {tool.name}")

print("\\nGitHub Operations:")
for tool in github_tools:
    print(f"  - {tool.name}")

# Create enhanced agent
try:
    # Bind all tools to LLM
    enhanced_llm = llm.bind_tools(all_tools)

    # Create new workflow for enhanced agent
    enhanced_workflow = StateGraph(AgentState)

    # Enhanced assistant node
    def enhanced_call_model(state: AgentState):
        """Enhanced assistant with GitHub capabilities"""
        print("🤖 Enhanced Assistant is processing...")
        messages = state["messages"]

        try:
            response = enhanced_llm.invoke(messages)
            print(f"💭 Enhanced response generated")
            return {"messages": [response]}
        except Exception as e:
            print(f"❌ Error in enhanced model: {e}")
            error_response = AIMessage(
                content=f"I encountered an error: {str(e)}. I can help with math operations and GitHub repository queries."
            )
            return {"messages": [error_response]}

    # Create enhanced tool node
    enhanced_tool_node = ToolNode(all_tools)

    # Build enhanced workflow
    enhanced_workflow.add_node("assistant", enhanced_call_model)
    enhanced_workflow.add_node("tools", enhanced_tool_node)
    enhanced_workflow.set_entry_point("assistant")

    enhanced_workflow.add_conditional_edges(
        "assistant",
        tools_condition,
        {"tools": "tools", "__end__": END}
    )
    enhanced_workflow.add_edge("tools", "assistant")

    # Compile enhanced agent
    enhanced_agent = enhanced_workflow.compile(checkpointer=MemorySaver())
    print("✅ Enhanced agent created successfully!")

except Exception as e:
    print(f"❌ Error creating enhanced agent: {e}")
    enhanced_agent = None

# Test the enhanced agent
def test_enhanced_agent(query: str, thread_id: str = "enhanced-test"):
    """Test the enhanced agent with GitHub and math capabilities"""
    print(f"🚀 Enhanced Test: '{query}'")
    print("=" * 60)

    if enhanced_agent is None:
        print("❌ Enhanced agent not available")
        return

    try:
        input_message = HumanMessage(content=query)
        config = {"configurable": {"thread_id": thread_id}}

        for chunk in enhanced_agent.stream({"messages": [input_message]}, config=config):
            print(f"📤 Enhanced Chunk: {chunk}")

    except Exception as e:
        print(f"❌ Error in enhanced agent test: {e}")

    print("=" * 60)
    print()

# Enhanced test cases
enhanced_test_queries = [
    "Add 25 and 35",
    "Get information about the repository 'microsoft/vscode'",
    "List the recent issues from 'facebook/react'",
    "Calculate 12 multiplied by 8, then tell me about the 'python/cpython' repository"
]

print("\\n🎯 Testing Enhanced Agent with GitHub + Math capabilities...")
print()

for i, query in enumerate(enhanced_test_queries, 1):
    print(f"Enhanced Test {i}:")
    test_enhanced_agent(query, f"enhanced-test-{i}")

print("🎉 Enhanced agent testing completed!")
print("\\n🏆 Agent Capabilities Summary:")
print("  ✅ Mathematical operations (add, multiply)")
print("  ✅ GitHub repository information")
print("  ✅ GitHub issues listing")
print("  ✅ Configurable LLM backend (Ollama/Gemini)")
print("  ✅ Conversation memory and state management")
print("  ✅ Extensible tool architecture")

## 🎯 Next Steps and Usage Instructions

### Configuration

1. **For Ollama (Local LLM)**:
   ```bash
   # Install Ollama: https://ollama.ai
   ollama pull llama2  # or your preferred model
   ```

2. **For Gemini (Cloud LLM)**:
   ```bash
   # Set your Google API key in .env
   GOOGLE_API_KEY=your_api_key_here
   DEFAULT_LLM_PROVIDER=gemini
   ```

3. **For GitHub Integration**:
   ```bash
   # Set your GitHub token in .env
   GITHUB_TOKEN=your_github_token_here
   GITHUB_REPO=your_username/your_repo
   ```

### Extending the Agent

You can easily add more tools:

```python
@tool
def your_custom_tool(param: str) -> str:
    \"\"\"Description of your tool\"\"\"
    # Your tool logic here
    return result

# Add to tools list and rebuild agent
```

### Production Deployment

- Consider using a more robust state store (Redis, PostgreSQL)
- Add error handling and retry logic
- Implement rate limiting for API calls
- Add logging and monitoring
- Use environment-specific configurations

### Architecture Benefits

- **Modular**: Easy to add/remove tools
- **Configurable**: Switch LLM providers easily  
- **Scalable**: LangGraph handles complex workflows
- **Maintainable**: Clear separation of concerns
- **Extensible**: Built for future enhancements

**🚀 Your autonomous GitHub MCP agent is ready to use!**